In [1]:
!pip install --upgrade git+git://github.com/glemercier/nvcc4jupyter.git
%reload_ext nvcc_plugin

  Cloning git://github.com/glemercier/nvcc4jupyter.git to /tmp/pip-req-build-ajygr2cb
  Running command git clone -q git://github.com/glemercier/nvcc4jupyter.git /tmp/pip-req-build-ajygr2cb
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4348 sha256=fa74afea00c9614873a45612cfa03b3363cd01e802d88170b68646f6288780f7
  Stored in directory: /tmp/pip-ephem-wheel-cache-tzo1933f/wheels/5e/23/ce/1e8885fe85a348e1208703a748142f57551cbefc18e4e05537
Successfully built NVCCPlugin
  Found existing installation: NVCCPlugin 0.0.2
    Uninstalling NVCCPlugin-0.0.2:
      Successfully uninstalled NVCCPlugin-0.0.2
directory /content/src already exists
Out bin /content/result.out


In [2]:
!mkdir -p /tmp/tp6
!wget --quiet https://github.com/glemercier/nvcc4jupyter/releases/download/tp6/data1.bin -P /tmp/tp6
!wget --quiet https://github.com/glemercier/nvcc4jupyter/releases/download/tp6/data2.bin -P /tmp/tp6
!wget --quiet https://github.com/glemercier/nvcc4jupyter/releases/download/tp6/data3.bin -P /tmp/tp6
!wget --quiet https://github.com/glemercier/nvcc4jupyter/releases/download/tp6/data4.bin -P /tmp/tp6
!wget --quiet https://github.com/glemercier/nvcc4jupyter/releases/download/tp6/image-edf.jpg -P /tmp/tp6

In [9]:
%%cu
#include <iostream>
#include <algorithm>                                                                                                    
#include <numeric>                                                                                                      
#include <cmath>
#include <cassert>
#include <vector>
#include <chrono>
#include <opencv2/opencv.hpp>
#include <cuda_runtime_api.h>

#define CUDA_ASSERT(x)  assert(x == cudaSuccess)

std::vector<float> loadData(const std::string &fileName);
bool compareImages(const cv::Mat &a, const cv::Mat &b);

//# Compute the distances between inputs and store them ino the 'distance' array. 'size' is the number                   
//# of elements in the arrays (here it's 512):                                                                           
//#                                                                                                                     
//# distance[0] = distance(a, b)                                                                                         
//# distance[1] = distance(a, c)                                                                                         
//# distance[2] = distance(a, d)                                                                                         
//#                                                                                                                     
//# The distance between two arrays a and b  is computed as per the formula:                                             
//#  distance = square_root(sum(pow2(a - b)))                                                                           
//#                                                                                                                     
//# See 'distanceCpu' for CPU implementation
__global__ void distanceCudaKernel(const float *a, float *b, float *sum, int size) {
    int idx = threadIdx.x + (blockIdx.x * blockDim.x);

    __shared__ unsigned int sum;

    // # one of the threads will initialize sum to 0
    if(idx == 0) {
        sum = 0;
    }

    // # Sync all thread
    if(idx < size){
        float dis = powf(a[idx] - b[idx], 2.0f);

        // # add dis to sum
        sum += dis;
    }
}

void distanceGpu(const float *a, const float *b, const float *c, float *d, float *sum, int size) {
  //# To be implemented ...   
  const int THREADS_PER_BLOCK = 128; // # Can be fine-tuned for better performance
  const int NUMBER_OF_BLOCKS = (size + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK; // # Round Up
  distanceCudaKernel<<<NUMBER_OF_BLOCKS, THREADS_PER_BLOCK>>>(a, b, sum, size);

  sum => array of 512 floats;
  sumCudaKernel<<<1, 1>>>();                      
}

//# Cast the pixels contained in 'inputImage' from uint8_t to float and normalize the R,G,B components                   
//# of each pixel around 0.5 as per the following formula: a = ((a / 255)  - 0.5) / 0.5                                  
//# Return the resulting image as a pointer to a GPU buffer allocated by the function.                                   
//# 'size' is the number of pixels * 3 channels of the image, or the number of total elements stored                     
//# in the array. See 'preprocessCpu' for CPU implementation.    

__global__ void preprocessCudaKernel(const uint8_t *in, float *out, int size) {
    // # Get the index corresponding to the computation unit where this kernel is running
    int idx = threadIdx.x + (blockIdx.x * blockDim.x);

    if(idx < size){
        out[idx] = (float)in[idx];
    }
}

void preprocessGpu(const uint8_t *inputImage, int size, float *out) {
  //# To be implemented...
  const int NUMBER_OF_BLOCKS = 128; // # Can be fine-tuned for better performance
  const int THREADS_PER_BLOCK = (NUMBER_OF_BLOCKS + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK; // # Round Up
  preprocessCudaKernel<<<NUMBER_OF_BLOCKS, THREADS_PER_BLOCK>>>(inputImage, out, size);
}

cv::Mat preprocessCpu(const cv::Mat &inputImage) {
  cv::Mat out;   
  inputImage.convertTo(out, CV_32F);

  out.forEach<cv::Point3_<float>>([](cv::Point3_<float> &p, const int pos[]) -> void {                                                  
    p.x = ((p.x / 255.0f) - 0.5f) / 0.5f;
    p.y = ((p.y / 255.0f) - 0.5f) / 0.5f; 
    p.z = ((p.z / 255.0f) - 0.5f) / 0.5f;                                                                                     
  });

  return out;
}

inline float distanceCpu(const std::vector<float>& a, const std::vector<float>& b) {
  float dis = 0.0f;
  for (int i = 0; i < a.size(); i++) {
    dis += std::pow(a[i] - b[i], 2);
  }
  return std::sqrt(dis);   
}

std::array<float, 3> distanceCpu(const std::vector<float>& a, const std::vector<float>& b,
      const std::vector<float>& c, const std::vector<float>& d) {
  return {
    distanceCpu(a, b),
    distanceCpu(a, c),
    distanceCpu(a, d)
  };                                                                                      
}

template <typename T>
T* CopyToGpuMem(const std::vector<T> &vec) {
  T *ptr;
  CUDA_ASSERT(cudaMalloc(&ptr, vec.size() * sizeof(T)));
  CUDA_ASSERT(cudaMemcpy(ptr, vec.data(), vec.size() * sizeof(T), cudaMemcpyHostToDevice));
  return ptr;
}

template <typename T>
T* CopyToGpuMem(const cv::Mat &mat) {
  T *ptr;
  CUDA_ASSERT(cudaMalloc(&ptr, mat.total() * mat.channels() * sizeof(T)));
  CUDA_ASSERT(cudaMemcpy(ptr, mat.data, mat.total() * mat.channels() * sizeof(T), cudaMemcpyHostToDevice));
  return ptr;
}

int main() {
  // Load input data
  std::vector<float> a = loadData("/tmp/tp6/data1.bin");
  std::vector<float> b = loadData("/tmp/tp6/data2.bin");
  std::vector<float> c = loadData("/tmp/tp6/data3.bin");
  std::vector<float> d = loadData("/tmp/tp6/data4.bin");
  assert(a.size() == b.size());
  assert(a.size() == c.size());
  assert(a.size() == d.size());
  cv::Mat image = cv::imread("/tmp/tp6/image-edf.jpg");
  assert(image.data != nullptr);
  assert(image.type() == CV_8UC3);

  // Copy data to GPU memory
  float *gpuA = CopyToGpuMem(a);
  float *gpuB = CopyToGpuMem(b);
  float *gpuC = CopyToGpuMem(c);
  float *gpuD = CopyToGpuMem(d);
  uint8_t *imageGpu = CopyToGpuMem<uint8_t>(image);

  auto start = std::chrono::high_resolution_clock::now();
  cv::Mat preprocessedCpu = preprocessCpu(image);
  auto stop = std::chrono::high_resolution_clock::now();
  int processingTimeCpu = std::chrono::duration_cast<std::chrono::microseconds>(stop - start).count();
  std::cout << "Preprocessing (CPU) - processing time: " << processingTimeCpu << " us" << std::endl;     

  float *preprocessedGpu = nullptr;
  CUDA_ASSERT(cudaMalloc(&preprocessedGpu, image.total() * image.channels() * sizeof(float)));

  start = std::chrono::high_resolution_clock::now();
  preprocessGpu(imageGpu, image.total() * image.channels(), preprocessedGpu);
  stop = std::chrono::high_resolution_clock::now();
  int processingTimeGpu = std::chrono::duration_cast<std::chrono::microseconds>(stop - start).count();
  std::cout << ((processingTimeGpu < processingTimeCpu) ? "\033[1m\033[32m" : "\033[1m\033[31m");
  std::cout << "Preprocessing (GPU) - processing time: " << processingTimeGpu << " us" << "\033[0m" << std::endl;

  // Convert GPU to cv::Mat and compare results
  cv::Mat preprocessedGpuMat(image.rows, image.cols, CV_32FC3);
  CUDA_ASSERT(cudaMemcpy(preprocessedGpuMat.ptr<float>(), preprocessedGpu,
      preprocessedGpuMat.total() * preprocessedGpuMat.channels() * sizeof(float),
      cudaMemcpyDeviceToHost));
  assert(compareImages(preprocessedGpuMat, preprocessedCpu));

  start = std::chrono::high_resolution_clock::now();
  auto disCpu = distanceCpu(a, b, c, d);
  stop = std::chrono::high_resolution_clock::now();
  processingTimeCpu = std::chrono::duration_cast<std::chrono::microseconds>(stop - start).count();
  std::cout << "Distance (CPU) = " << disCpu[0] << ", " << disCpu[1] << ", " << disCpu[2]
        << " - processing time: " << processingTimeCpu << " us" << std::endl;      

  // Allocate intermediate GPU buffers
  float *distance;
  CUDA_ASSERT(cudaMalloc(&distance, 3 * sizeof(float)));

  start = std::chrono::high_resolution_clock::now();
  distanceGpu(gpuA, gpuB, gpuC, gpuD, distance, a.size());
  stop = std::chrono::high_resolution_clock::now();

  float disGpu[3];
  CUDA_ASSERT(cudaMemcpy(disGpu, distance, sizeof(disGpu), cudaMemcpyDeviceToHost));
  processingTimeGpu = std::chrono::duration_cast<std::chrono::microseconds>(stop - start).count();
  std::cout << ((processingTimeGpu < processingTimeCpu) ? "\033[1m\033[32m" : "\033[1m\033[31m");
  std::cout << "Distance (GPU) = " << disGpu[0] << ", " << disGpu[1] << ", " << disGpu[2]
        << " - processing time: " << processingTimeGpu << " us" << "\033[0m" << std::endl;

  float epsilon = 10e-4;
  assert(fabs(disCpu[0] - disGpu[0]) < epsilon);
  assert(fabs(disCpu[1] - disGpu[1]) < epsilon);
  assert(fabs(disCpu[2] - disGpu[2]) < epsilon);

  cudaFree(gpuA);
  cudaFree(gpuB);
  cudaFree(gpuC);
  cudaFree(imageGpu);
  cudaFree(preprocessedGpu);
  cudaFree(distance);
}

std::vector<float> loadData(const std::string &fileName) {
  // open the file:
  std::ifstream file(fileName, std::ios::binary);
  assert(file.is_open());

  // get its size:
  file.seekg(0, std::ios::end);
  std::size_t fileSize = file.tellg();
  file.seekg(0, std::ios::beg);

  // read the data:
  std::vector<float> fileData(fileSize / sizeof(float));
  file.read(reinterpret_cast<char *>(fileData.data()), fileData.size() * sizeof(float));
  return fileData;
}

bool compareImages(const cv::Mat &a, const cv::Mat &b) {
  cv::Mat diff;
  cv::subtract(a, b, diff);
  cv::Mat channels[3];
  cv::split(diff, channels);
  return (cv::countNonZero(channels[0]) == 0)
      && (cv::countNonZero(channels[1]) == 0)
      && (cv::countNonZero(channels[2]) == 0);
}

/tmp/tmp8yprurzy/52561627-bf72-4ae6-98ed-366ce90ab4d7.cu(30): error: "sum" has already been declared in the current scope

/tmp/tmp8yprurzy/52561627-bf72-4ae6-98ed-366ce90ab4d7.cu(50): error: argument of type "const float *" is incompatible with parameter of type "float *"

/tmp/tmp8yprurzy/52561627-bf72-4ae6-98ed-366ce90ab4d7.cu(52): error: expected an expression

/tmp/tmp8yprurzy/52561627-bf72-4ae6-98ed-366ce90ab4d7.cu(53): error: identifier "sumCudaKernel" is undefined

/tmp/tmp8yprurzy/52561627-bf72-4ae6-98ed-366ce90ab4d7.cu(74): warning: variable "THREADS_PER_BLOCK" is used before its value is set

4 errors detected in the compilation of "/tmp/tmpxft_000004cb_00000000-8_52561627-bf72-4ae6-98ed-366ce90ab4d7.cpp1.ii".

